# Mapserver / Mapfile

In [1]:
from AlertaDengue.dados import dbdata
import os
import sqlalchemy as sqla
# local
import AlertaDengue as alert_dengue

In [2]:
def get_passwd_from_ini(filepath: str):
    with open(filepath) as f:
        text = f.read()
        i = text.index('PSQL_PASSWORD')
        f = text[i:].index('\n')
        passwd = text[i:i+f].split('=')
        return passwd[1].strip()

In [3]:
# mapfile templates

mapfile_template = '''
MAP
    IMAGETYPE      PNG
    EXTENT      -87.93749190049996 -20.493952297946063 -14.892815463099353  7.223000268026496
 
    SIZE           1800 1600
    IMAGECOLOR     255 255 255
    CONFIG "MS_ERRORFILE" "%(ms_error_path)s"
    CONFIG "CPL_DEBUG" "ON"
    CONFIG "PROJ_DEBUG" "ON"
    DEBUG 5

    PROJECTION
      "init=epsg:3857"
    END

    WEB
      METADATA
        "wms_title" "Alerta Dengue"
        "wms_onlineresource" "%(ms_cgi_path)s?"
        "wms_enable_request" "*"
        "wms_srs" "EPSG:3857"
        "wms_feature_info_mime_type" "text/html"
        "wms_format" "image/png"
      END
    END
 
    %(layers)s
 
END
'''

mapfile_layer_template = '''
    LAYER
        NAME         "%(city_name)s"
        DATA         "%(shp_path)s/%(geocode)s"
        STATUS       default
        TYPE         POLYGON
 
        CLASS
            STYLE
                COLOR %(rgb)s
            END
        END
    END
'''

In [4]:
# set connection

db = 'dengue'
user = 'dengueadmin'
host = 'localhost'
passwd = get_passwd_from_ini('../AlertaDengue/AlertaDengue/settings.ini')
dsn = "postgresql://{}:{}@{}/{}".format(
    user, passwd, host, db
)

engine = sqla.create_engine(dsn)

In [5]:
shp_path = '%s/static/shapefile' % alert_dengue.__path__[0]
ms_error_path = '/home/xmn/alerta_dengue_ms.log'
ms_cgi_path = 'http://localhost:81/cgi-bin/mapserv'
ms_mapfile = '%s/mapserv/alerta_dengue.map' % (
    os.path.dirname(alert_dengue.__path__[0])
)

In [6]:
sql = '''
SELECT geocodigo, nome, uf 
FROM "Dengue_global"."Municipio" 
WHERE uf in (
  'Paraná', 'Rio de Janeiro', 'Ceará', 'Minas Gerais', 'Espírito Santo')
ORDER BY uf, nome;
'''

In [7]:
rgb = {
    'Ceará': '255 0 0',
    'Espírito Santo': '0 255 0',
    'Minas Gerais': '0 0 255',
    'Rio de Janeiro': '255 255 0',
    'Paraná': '255 0 255',
}

In [11]:
state = 'RJ'
disease = 'dengue'

cities_alert = dbdata.NotificationResume.get_cities_alert_by_state(
    state, disease
)

alerts = dict(
    cities_alert[['municipio_geocodigo', 'level_alert']].values
)

alerts

{}

In [8]:
layers = ''

with engine.connect() as conn:
    result = conn.execute(sql).fetchall()
    for geocode, city_name, state_country in result:
        layer = mapfile_layer_template % {
            'geocode': geocode,
            'city_name': city_name,
            'rgb': rgb[state_country],
            'shp_path': shp_path
        }
        
        if not os.path.exists('%s/%s.shp' % (shp_path, geocode)):
            layer = layer.replace('\n', '\n# ') + '\n'
        
        layers += layer

ms_config = {
    'layers': layers,
    'ms_error_path': ms_error_path,
    'ms_cgi_path': ms_cgi_path
}
mapfile = mapfile_template % ms_config

with open(ms_mapfile, 'w') as f:
    f.write(mapfile)
    

In [13]:
%%html

<div id="alerta_dengue" style="height: 200px; width: 200px;"></div>

<link rel="stylesheet" 
 href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/1.2.0/leaflet.css" />
<script type="text/javascript" 
 src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/1.2.0/leaflet.js">
</script>

In [14]:
%%javascript

var mapfile_url = (
    'http://localhost:81/cgi-bin/mapserv?' + 
    'map=/var/www/mapserver/mapserv/alerta_dengue.map&' + 
    'mode=map'
);

var map = L.map('alerta_dengue').setView([51.4, -1.25], 7); 

L.tileLayer.wms(mapfile_url, {
    layers: 'Roads',
    format: 'image/png',
    transparent: true,
    attribution: "Dan's Amazing Roads",
    maxZoom: 18,
    minZoom: 5,
}).addTo(map);

<IPython.core.display.Javascript object>